# Import packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt import expected_returns
from datetime import datetime
from pandas.tseries.offsets import BDay

In [3]:
import time
import pickle

In [4]:
fund_df = pd.read_csv('data_processor_update/sp500_fundamental_199601_202502.csv')

In [5]:
fund_df.drop_duplicates('gvkey', inplace=True)
fund_df = fund_df[['gvkey','tic']]

# 1. Read Input Data

In [6]:
df_price = pd.read_csv("data_processor_update/sp500_price_199601_202502.csv")

In [7]:
df_price.shape

(9029820, 22)

In [8]:
df_price.columns

Index(['tic', 'cusip', 'permno', 'permco', 'issuno', 'hexcd', 'hsiccd', 'date',
       'bidlo', 'askhi', 'prc', 'adj_close_q', 'vol', 'ret', 'bid', 'ask',
       'shrout', 'cfacpr', 'cfacshr', 'openprc', 'numtrd', 'retx'],
      dtype='object')

In [9]:
# df_price['adj_price'] = df_price['prccd'] / df_price['ajexdi']
df_price['adj_price'] = df_price['adj_close_q']
df_price = pd.merge(df_price, fund_df, on='tic')
# df_price['gvkey'] = df_price['tic']
df_price['datadate'] = df_price['date']

In [10]:
df_price['datadate'] = df_price['datadate'].str[:10]

In [11]:
df_price = df_price[["gvkey", "datadate", 'adj_price']]

In [12]:
len(df_price.gvkey.unique())

741

In [14]:
selected_stock = pd.read_csv("stock_selected.csv",index_col=0)

In [15]:
selected_stock=selected_stock[selected_stock.trade_date>='2018-03-01'].reset_index(drop=True)

In [16]:
selected_stock.shape

(3784, 3)

In [17]:
selected_stock.head()

,gvkey,predicted_return,trade_date
0,21212,0.020121,2018-03-01
1,178707,0.025243,2018-03-01
2,178917,0.025753,2018-03-01
3,176622,0.026587,2018-06-01
4,178707,0.026557,2018-06-01


# 2. Get trade date

In [18]:
print("Number of unique stocks selected: ", len(selected_stock.gvkey.unique()))

Number of unique stocks selected:  607


In [19]:
all_date=df_price.datadate.unique()

In [20]:
len(all_date)

7325

In [21]:
trade_date=selected_stock.trade_date.unique()

In [22]:
trade_date

array(['2018-03-01', '2018-06-01', '2019-03-01', '2020-06-01',
       '2020-09-01', '2020-12-01', '2021-03-01', '2021-06-01',
       '2021-09-01', '2021-12-01', '2022-03-01', '2022-06-01',
       '2022-09-01', '2022-12-01', '2023-03-01', '2023-06-01',
       '2023-09-01', '2023-12-01', '2024-03-01'], dtype=object)

In [23]:
print("Number of trade dates", len(trade_date))

Number of trade dates 19


# 3. Get daily 1 year return table in each 89 trade period

In [24]:
selected_stock.head()

,gvkey,predicted_return,trade_date
0,21212,0.020121,2018-03-01
1,178707,0.025243,2018-03-01
2,178917,0.025753,2018-03-01
3,176622,0.026587,2018-06-01
4,178707,0.026557,2018-06-01


In [25]:
# took about 9 minutes to run
start = time.time()
all_return_table={}
#all_predicted_return={}
all_stocks_info = {}
#for i in range(0,1):
for i in range(len(trade_date)):
    #match trading date
    index = selected_stock.trade_date==trade_date[i]
    #get the corresponding trade period's selected stocks' name
    stocks_name=selected_stock.gvkey[selected_stock.trade_date==trade_date[i]].values
    temp_info = selected_stock[selected_stock.trade_date==trade_date[i]]
    temp_info = temp_info.reset_index()
    del temp_info['index']
    all_stocks_info[trade_date[i]] = temp_info
    #get the corresponding trade period's selected stocks' predicted return
    asset_expected_return=selected_stock[index].predicted_return.values
    
    
    #determine the business date
    #print(convert_to_yyyymmdd)
    tradedate = pd.to_datetime(trade_date[i])
    ts = datetime(tradedate.year-1, tradedate.month, tradedate.day)
    bd = pd.tseries.offsets.BusinessDay(n =1) 
    new_timestamp = ts - bd 
    all_date = pd.to_datetime(all_date, format="%Y-%m-%d")
    get_date_index=(all_date<tradedate) & (all_date>new_timestamp)
    get_date=all_date[get_date_index]
    #get adjusted price table
    return_table=pd.DataFrame()
    for m in range(len(stocks_name)):
        #get stocks's name
        index_tic=(df_price.gvkey==stocks_name[m])
        #get this stock's all historicall price from sp500_price
        sp500_temp=df_price[index_tic]
        merge_left_data_table = pd.DataFrame(get_date)
        merge_left_data_table.columns = ['datadate']
        #print(merge_left_data_table)
        sp500_temp.datadate = pd.to_datetime(sp500_temp.datadate, format="%Y-%m-%d")
        temp_price=merge_left_data_table.merge(sp500_temp, on=['datadate'], how='left')
        
        temp_price = temp_price.dropna()
        temp_price['daily_return']=temp_price.adj_price.pct_change()
        # return_table=return_table.append(temp_price,ignore_index=True)
        return_table = pd.concat([return_table,temp_price])
    all_return_table[trade_date[i]] = return_table
end = time.time()
print("Time consuming: ", (end-start)/60, " minutes")

Time consuming:  0.39164224863052366  minutes


In [26]:
check=all_stocks_info['2018-03-01'].gvkey.unique()

In [27]:
for i in check:
    if i not in all_return_table['2018-03-01'].gvkey.unique():
        print(i)

In [28]:
df_price.head()

,gvkey,datadate,adj_price
0,126554,1999-11-18,26.511744
1,126554,1999-11-19,24.327530
2,126554,1999-11-22,26.511744
3,126554,1999-11-23,24.101572
4,126554,1999-11-24,24.741783


In [29]:
selected_stock[selected_stock.gvkey==34443]

,gvkey,predicted_return,trade_date
428,34443,0.047896,2022-03-01
439,34443,0.052521,2022-06-01
470,34443,0.057469,2023-03-01


In [30]:
df_price[df_price.gvkey==34443]

,gvkey,datadate,adj_price
1113684,34443,2019-03-20,36.684483
1113685,34443,2019-03-21,36.080452
1113686,34443,2019-03-22,35.800514
1113687,34443,2019-03-25,36.205669
1113688,34443,2019-03-26,35.984673
...,...,...,...
5260508,34443,2025-02-03,37.380001
5260509,34443,2025-02-04,38.549999
5260510,34443,2025-02-05,37.950001
5260511,34443,2025-02-06,38.770000


## Save to pickle

In [31]:
with open('all_return_table.pickle', 'wb') as handle: 
    pickle.dump(all_return_table, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [32]:
with open('all_stocks_info.pickle', 'wb') as handle:
    pickle.dump(all_stocks_info, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [33]:
with open('all_return_table.pickle', 'rb') as handle:
    all_return_table = pickle.load(handle)

with open('all_stocks_info.pickle', 'rb') as handle:
    all_stocks_info = pickle.load(handle)


In [34]:
len(all_stocks_info)

19

In [35]:
all_return_table['2018-03-01']

,datadate,gvkey,adj_price,daily_return
0,2017-03-01,21212.0,27.463104,NaN
1,2017-03-01,21212.0,27.463104,0.000000e+00
2,2017-03-02,21212.0,26.935143,-1.922440e-02
3,2017-03-02,21212.0,26.935143,0.000000e+00
4,2017-03-03,21212.0,27.208225,1.013853e-02
...,...,...,...,...
499,2018-02-26,110179.0,37.220360,0.000000e+00
500,2018-02-27,110179.0,35.885040,-3.587605e-02
501,2018-02-27,110179.0,35.885044,1.063033e-07
502,2018-02-28,110179.0,35.452156,-1.206319e-02


In [36]:
len(all_return_table)

19

In [37]:
all_stocks_info['2018-03-01']

,gvkey,predicted_return,trade_date
0,21212,0.020121,2018-03-01
1,178707,0.025243,2018-03-01
2,178917,0.025753,2018-03-01
3,4430,0.025999,2018-03-01
4,6788,0.036743,2018-03-01
...,...,...,...
155,30490,0.042709,2018-03-01
156,32580,0.057244,2018-03-01
157,62374,0.042421,2018-03-01
158,65640,0.042267,2018-03-01


# 4. Potfolio Optimization using pypfopt

In [38]:
# took under 5 minutes to run
from pypfopt import objective_functions
stocks_weight_table = pd.DataFrame([])

for i in range(len(trade_date)):
    # get selected stocks information
    p1_alldata=(all_stocks_info[trade_date[i]])
    # sort it by tic
    p1_alldata=p1_alldata.sort_values('gvkey')
    p1_alldata = p1_alldata.reset_index()
    del p1_alldata['index']
    
    print(p1_alldata)
    # get selected stocks tic
    p1_stock = p1_alldata.gvkey
    
    # get predicted return from selected stocks
    p1_predicted_return=p1_alldata.pivot_table(index = 'trade_date',columns = 'gvkey', values = 'predicted_return')
    # use the predicted returns as the Expected returns to feed into the portfolio object
    

    # get the 1-year historical return
    p1_return_table=all_return_table[trade_date[i]]
    p1_return_table_pivot=p1_return_table.pivot_table(index = 'datadate',columns = 'gvkey', values = 'daily_return')
    print(p1_return_table)
    selected_stocks = list(set(p1_predicted_return.columns).intersection(p1_return_table_pivot.columns))
    # use the 1-year historical return table to calculate covariance matrix between selected stocks
    p1_predicted_return = p1_predicted_return.loc[:, selected_stocks]
    p1_return_table_pivot = p1_return_table_pivot.loc[:, selected_stocks]
    S = risk_models.sample_cov(p1_return_table_pivot)
    mu = p1_predicted_return.T.values
   # del S.index.name 
    # mean variance
    ef_mean = EfficientFrontier(mu, S,weight_bounds=(0, 0.05))
    # raw_weights_mean = ef_mean.max_sharpe()
    raw_weights_mean = ef_mean.nonconvex_objective(
        objective_functions.sharpe_ratio,
        objective_args = (ef_mean.expected_returns, ef_mean.cov_matrix),
        weights_sum_to_one = True
    )
    cleaned_weights_mean = ef_mean.clean_weights()
    # print(f'output{cleaned_weights_mean}')
    #print(raw_weights_mean)
    #ef.portfolio_performance(verbose=True)
    #print("cleaned_weights_mean: ", cleaned_weights_mean)

    # minimum variance
    ef_min = EfficientFrontier([0]*len(mu), S,weight_bounds=(0, 0.05))
   # raw_weights_min = ef_min.max_sharpe()
    raw_weights_min = ef_min.nonconvex_objective(
        objective_functions.sharpe_ratio,
        objective_args = (ef_min.expected_returns, ef_min.cov_matrix),
        weights_sum_to_one = True
    )
    cleaned_weights_min = ef_min.clean_weights()
    #print(cleaned_weights_min)
    print("cleaned_weights_min: ", cleaned_weights_min)

    idx = np.isin(p1_alldata.gvkey, selected_stocks)
    p1_alldata["mean_weight"] = 0
    p1_alldata.loc[idx, 'mean_weight'] = list(cleaned_weights_mean.values())

    p1_alldata["min_weight"] = 0
    p1_alldata.loc[idx, 'min_weight'] = list(cleaned_weights_min.values())

    p1_alldata["equal_weight"] = 0
    p1_alldata.loc[idx, 'equal_weight'] = np.ones(len(cleaned_weights_mean)) / len(cleaned_weights_mean)
    # p1_alldata["mean_weight"] = 0
    # p1_alldata['mean_weight'][idx] = list(cleaned_weights_mean.values())
    # p1_alldata["min_weight"] = 0
    # p1_alldata['min_weight'][idx] = list(cleaned_weights_min.values())
    # p1_alldata["equal_weight"] = 0
    # p1_alldata['equal_weight'][idx] = np.ones(len(cleaned_weights_mean.values())) / len(cleaned_weights_mean.values())
    #ef.portfolio_performance(verbose=True)
    
    stocks_weight_table = pd.concat([stocks_weight_table, p1_alldata])
    print(trade_date[i], ": Done")


      gvkey  predicted_return  trade_date
0      1045          0.034466  2018-03-01
1      1075          0.036502  2018-03-01
2      1161          0.032784  2018-03-01
3      1230          0.035069  2018-03-01
4      2220          0.025938  2018-03-01
..      ...               ...         ...
155  184500          0.031506  2018-03-01
156  184725          0.033817  2018-03-01
157  184996          0.039749  2018-03-01
158  186278          0.033225  2018-03-01
159  312009          0.040051  2018-03-01

[160 rows x 3 columns]
      datadate     gvkey  adj_price  daily_return
0   2017-03-01   21212.0  27.463104           NaN
1   2017-03-01   21212.0  27.463104  0.000000e+00
2   2017-03-02   21212.0  26.935143 -1.922440e-02
3   2017-03-02   21212.0  26.935143  0.000000e+00
4   2017-03-03   21212.0  27.208225  1.013853e-02
..         ...       ...        ...           ...
499 2018-02-26  110179.0  37.220360  0.000000e+00
500 2018-02-27  110179.0  35.885040 -3.587605e-02
501 2018-02-27  110179

In [39]:
stocks_weight_table.shape

(3784, 6)

In [40]:
stocks_weight_table.head()

,gvkey,predicted_return,trade_date,mean_weight,min_weight,equal_weight
0,1045,0.034466,2018-03-01,0.00625,0.00625,0.00625
1,1075,0.036502,2018-03-01,0.00625,0.00625,0.00625
2,1161,0.032784,2018-03-01,0.00625,0.00625,0.00625
3,1230,0.035069,2018-03-01,0.00625,0.00625,0.00625
4,2220,0.025938,2018-03-01,0.00625,0.00625,0.00625


## save to excel or csv

In [41]:
df_mean = pd.DataFrame()
df_mean['trade_date'] = stocks_weight_table['trade_date']
df_mean['gvkey'] = stocks_weight_table['gvkey']
df_mean['weights'] = stocks_weight_table['mean_weight']
df_mean['predicted_return'] = stocks_weight_table['predicted_return']
df_mean.to_excel("mean_weighted.xlsx")

In [42]:
df_min = pd.DataFrame()
df_min['trade_date'] = stocks_weight_table['trade_date']
df_min['gvkey'] = stocks_weight_table['gvkey']
df_min['weights'] = stocks_weight_table['min_weight']
df_min['predicted_return'] = stocks_weight_table['predicted_return']
df_min.to_excel("minimum_weighted.xlsx")

In [43]:
df_equal = pd.DataFrame()
df_equal['trade_date'] = stocks_weight_table['trade_date']
df_equal['gvkey'] = stocks_weight_table['gvkey']
df_equal['weights'] = stocks_weight_table['equal_weight']
df_equal['predicted_return'] = stocks_weight_table['predicted_return']
df_equal.to_excel("equally_weighted.xlsx")